In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pandas as pd


import time



In [2]:
options = Options()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)
driver.get('https://claro.qualtrics.com/Q/MyProjectsSection')


In [3]:
# Preenche login
driver.find_element(By.ID, 'username').send_keys('93239859')
driver.find_element(By.ID, 'password').send_keys('N1nh@*25')
driver.find_element(By.ID, "signOnButton").click()

In [4]:
logado = False
while not logado:
    try:
        elemento = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Projetos e programas")))
        logado = True
    except:
        pin = elemento = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div[1]/div/div[2]/div[3]")))
        pin = pin.text.strip()  



In [5]:
driver.get('https://claro.qualtrics.com/reporting-dashboard/web/6616d90f5c152200082e2233/pages/Page_4bccad9e-f0d7-4265-8a8c-22b831245106/view')


In [6]:
filtroEps = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/main/div[2]/div[1]/div[1]/div[1]/div/div/div/ul/li[2]/span/span/span[2]/span[15]/span/div/div/div/div/div/div/div/div/div/span/button/span[2]')))
filtroEps.click()
TEL_CENTRO = 'sub0_23'
TEL_TELEMATICA = 'sub0_24'
TELEMATICA_FEIRADESATANA_DACO1 = 'sub0_25'

tel_centro = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(., 'TEL CENTRO')]")
    )
)
tel_centro.click()

tel_telematica = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(., 'TEL TELEMATICA')]")
    )
)
tel_telematica.click()

tel_telematicadac01 = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(., 'TELEMATICA|FEIRADESANTANA|DAC01')]")
    )
)
tel_telematicadac01.click()






In [9]:
filtroPeriodo = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/main/div[2]/div[1]/div[1]/div[1]/div/div/div/ul/li[2]/span/span/span[2]/span[2]/span/div/div/span/button/span[2]')))
filtroPeriodo.click()

botao_periodo = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//button[@aria-label='Data Resposta seleção Este mês']")
    )
)
botao_periodo.click()

periodoUltimos7Dias = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, 'sub0_3')))

periodoUltimos7Dias.click()


In [20]:
xpath_btn = '/html/body/div[1]/div[4]/main/div[2]/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/div[3]/div/ng-transclude/div/dashboards.core.layout-engine/div/div/div/div/div/div[14]/div/div/div/div/div/div/div[2]/div/div/div/div[1]/h3/div/div[2]/div/div/button/div[1]'

found = False
for i in range(50):
    try:
        element = driver.find_element(By.XPATH, xpath_btn)
        driver.execute_script("arguments[0].scrollIntoView(true);", element)

        # Espera até estar clicável
        element = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, xpath_btn))
        )

        driver.execute_script("arguments[0].click();", element)
        found = True
        break

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("document.scrollingElement.scrollBy(0, 600);")
        time.sleep(1)

if not found:
    raise Exception("❌ Elemento não encontrado após 50 tentativas")


In [21]:

qtd_feedback  = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[5]/main/div[2]/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/div[3]/div/ng-transclude/div/dashboards.core.layout-engine/div/div/div/div/div/div[14]/div/div/div/div/div/div/div[2]/div/div/div/div[1]/h3/div/div[2]/div/div/button/div[2]")))
qtd_feedback = qtd_feedback.text
print(qtd_feedback.strip())


12


In [35]:
import pandas as pd

todos_dados = []  # cada item será um registro de uma resposta

respostas = WebDriverWait(driver, 15).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.styles__response___\\+0-OI"))
)

for resposta in respostas:
    numero = resposta.find_element(By.CSS_SELECTOR, "div.styles__ticker-content___vHHyl").text.strip()
    aria_label = resposta.get_attribute("aria-label")
    print(f"Número: {numero} | Label: {aria_label}")

    try:
        # Localiza o botão de menu
        menu_btn = resposta.find_element(By.CSS_SELECTOR, "button[aria-label='Menu']")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", menu_btn)
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(menu_btn))
        driver.execute_script("arguments[0].click();", menu_btn)
        print(f"Cliquei no menu da resposta {numero}")

        # Clica em "Ver registo completo"
        ver_registro = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='Ver registo completo']"))
        )
        driver.execute_script("arguments[0].click();", ver_registro)

        # Espera o modal abrir
        modal = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='dialog']"))
        )

        # Busca o tbody interno relativo ao modal
        tbody = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, ".//div[@role='dialog']//table/tbody/tr/td/table/tbody")
            )
        )
        rows = tbody.find_elements(By.TAG_NAME, "tr")

        registro = {}  # aqui vamos montar o dict para esta resposta
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) == 2:
                campo = cells[0].text.strip()
                valor = cells[1].text.strip()
                registro[campo] = valor

        # Adiciona também o número e label da resposta
        registro["Número"] = numero
        registro["Label"] = aria_label

        todos_dados.append(registro)

        # Fecha o modal
        fechar_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[normalize-space()='Fechar' or text()='×']"))
        )
        fechar_btn.click()
        print(f"Fechei o registro da resposta {numero}")

    except Exception as e:
        print(f"Não consegui processar a resposta {numero}: {e}")

# Depois de processar todos os modais, monta o DataFrame
df_final = pd.DataFrame(todos_dados)
df_final.to_excel("respostas.xlsx", index=False, engine='openpyxl')
print("Arquivo 'respostas.xlsx' salvo com sucesso!")


Número: 0 | Label: Resposta: 1. DAC. RIO DE JANEIRO. há 17 horas. Liguei fiquei mais de 25 minutos em ligação para não ter solução,  eu quero um cancelamento , fiz todas autenticações,  sms, para no final ser informado que houve erro no sistema da Claro que tudo foi perdido e que só posso pedir o cancelamento daqui a 3 dias, um horror , estou profundamente chateado e insatisfeito
Cliquei no menu da resposta 0
Fechei o registro da resposta 0
Número: 3 | Label: Resposta: 2. 3. DAC. MANAUS. há 18 horas
Cliquei no menu da resposta 3
Fechei o registro da resposta 3
Número: 10 | Label: Resposta: 3. 10. DAC. RECIFE. há 19 horas. Bom atendimento 
Cliquei no menu da resposta 10
Fechei o registro da resposta 10
Número: 5 | Label: Resposta: 4. 5. DAC. CAPAO DA CANOA. há 20 horas. dificuldade de resolver o problema por telefone
Cliquei no menu da resposta 5
Fechei o registro da resposta 5
Número: 1 | Label: Resposta: 5. 1. DAC. PINDAMONHANGABA. há 21 horas. Estou tentando há tempos resolver o prob

In [34]:
# Depois de processar todos os modais
df_final = pd.concat(data, ignore_index=True, sort=False)
df_final.to_excel("respostas.xlsx", index=False, engine='openpyxl')

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid